In [1]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
import torchvision.transforms as transforms
import torch
from torch import nn
from torch.nn import functional as F
import warnings
warnings.filterwarnings("ignore")

In [2]:
class LeaveDataDataset(Dataset):
    def __init__(self,train_csv_path,mode='train',train_ratio=0.8,
                 resize_height=256,resize_width=256):
        self.resize_height = resize_height
        self.resize_width = resize_width
        self.train_ratio = train_ratio
        self.data = pd.read_csv(train_csv_path)
        self.mode = mode
        self.data_len = len(self.data)
        self.train_len = int(self.data_len * self.train_ratio)
        if mode == 'train':
            self.image = np.asarray(self.data.iloc[:self.train_len, 0])
            self.label = np.asarray(self.data.iloc[:self.train_len, 1])
        elif mode == 'valid':
            self.image = np.asarray(self.data.iloc[self.train_len:, 0])
            self.label = np.asarray(self.data.iloc[self.train_len:, 1])
        elif mode == 'test':
            self.image =np.asarray(self.data.iloc[:, 0])
        self.len = len(self.image)
        print('Finished reading the {} set of Leaves Dataset ({} samples found)'
              .format(mode, self.len))
    def __getitem__(self, index):
        image_path = self.image[index]
        image = Image.open(image_path)
        # if image.mode != 'L':
        #     image = image.convert('L')
        if self.mode == 'train':
            transform = transforms.Compose([transforms.Resize((224, 224)),
                                            transforms.RandomHorizontalFlip(p=0.5), 
                                            transforms.RandomVerticalFlip(p=0.5),
                                            transforms.ToTensor()])
        else:
            transform = transforms.Compose([transforms.Resize((224, 224)),
                                            transforms.ToTensor()])
        img = transform(image)
        if self.mode == 'test':
            return img
        else:
            label = self.label[index]
            label_num = class_to_num[label]
            return img,label_num
    def __len__(self):
        return self.len

In [3]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels,use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)
def resnet18(num_classes, in_channels=1):
    def resnet_block(in_channels, out_channels, num_residuals,
                     first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(Residual(in_channels, out_channels,use_1x1conv=True, strides=2))
            else:
                blk.append(Residual(out_channels, out_channels))
        return nn.Sequential(*blk)
    net = nn.Sequential(
        nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU())
    net.add_module("resnet_block1", resnet_block(64, 64, 2, first_block=True))
    net.add_module("resnet_block2", resnet_block(64, 128, 2))
    net.add_module("resnet_block3", resnet_block(128, 256, 2))
    net.add_module("resnet_block4", resnet_block(256, 512, 2))
    net.add_module("global_avg_pool", nn.AdaptiveAvgPool2d((1,1)))
    net.add_module("fc", nn.Sequential(nn.Flatten(),
                                       nn.Linear(512, num_classes)))
    return net

In [4]:
def train_leaf(net, num_epochs, lr, device,best_acc):
    print(device)
    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    net = net.to(device)    
    train_dataset = LeaveDataDataset(train_path,mode='train')
    val_dataset = LeaveDataDataset(train_path,mode='valid')

    train_iter = DataLoader(dataset=train_dataset,batch_size=batch_size, shuffle=True)
    val_iter = DataLoader(dataset=val_dataset,batch_size=batch_size, shuffle=False)
    
    for epoch in range(num_epochs):
        net.train()
        print('Train Epoch: {}/{}'.format(epoch, num_epochs),'train len: {}'.format(len(train_iter)))
        train_loss, train_acc, n = 0.0, 0.0, 0   
        for X, y in tqdm(train_iter):
            X = X.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            train_loss += l.item() * y.shape[0]
            train_acc += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        train_loss /= n
        train_acc /= n
        print(f"Epoch {epoch}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}")
        
        net.eval()
        print('Val Epoch: {}/{}'.format(epoch, num_epochs),'val len: {}'.format(len(val_iter)))
        val_loss, val_acc, m = 0.0, 0.0, 0
        for X, y in tqdm(val_iter):
            X = X.to(device)
            y = y.to(device)
            with torch.no_grad():
                y_hat = net(X)
            l = loss(y_hat, y)
            val_loss += l.item() * y.shape[0]
            val_acc += (y_hat.argmax(dim=1) == y).sum().item()
            m += y.shape[0]
        val_loss /= m
        val_acc /= m
        print(f"Epoch {epoch}: Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), './working/leaf_resnet18.pth')
            print('saving model with acc {:.3f}'.format(best_acc))

In [5]:
num_epochs = 20
batch_size=16
lr=0.05
best_acc = 0.1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_path = 'train.csv'
test_path = 'test.csv'
# data = pd.read_csv('train.csv')
# label_all = sorted(list(set(data.iloc[:,1])))
# n_classes = len(label_all)
# class_to_num = dict(zip(label_all,range(n_classes)))
# np.save('class_to_num.npy',class_to_num)
# num_to_class = dict(zip(range(n_classes),label_all))
# np.save('num_to_class.npy',num_to_class)
class_to_num = np.load('class_to_num.npy', allow_pickle=True).item()
num_to_class = np.load('num_to_class.npy', allow_pickle=True).item()
n_classes = len(class_to_num)
model = resnet18(num_classes = n_classes,in_channels=3)

In [ ]:
train_leaf(model, num_epochs, lr, device,best_acc)
# torch.save(model.state_dict(), './working/leaf_resnet18.pth')

cuda
Finished reading the train set of Leaves Dataset (14682 samples found)
Finished reading the valid set of Leaves Dataset (3671 samples found)
Train Epoch: 0/20 train len: 918


100%|██████████| 918/918 [25:51<00:00,  1.69s/it]


Epoch 0: Train Loss=4.5372, Train Acc=0.0535
Val Epoch: 0/20 val len: 230


100%|██████████| 230/230 [04:43<00:00,  1.23s/it]


Epoch 0: Val Loss=5.2052, Val Acc=0.0542
Train Epoch: 1/20 train len: 918


100%|██████████| 918/918 [25:01<00:00,  1.64s/it]


Epoch 1: Train Loss=3.6052, Train Acc=0.1391
Val Epoch: 1/20 val len: 230


100%|██████████| 230/230 [04:43<00:00,  1.23s/it]


Epoch 1: Val Loss=64.1542, Val Acc=0.0082
Train Epoch: 2/20 train len: 918


 43%|████▎     | 392/918 [10:59<14:57,  1.71s/it]

In [6]:
saveFileName = './resnet18_raw/submission.csv'
test_dataset = LeaveDataDataset(test_path, mode='test')
test_iter = DataLoader(dataset=test_dataset,batch_size=batch_size, shuffle=False)

model = model.to(device)
model.load_state_dict(torch.load('./working/leaf_resnet18.pth'))
model.eval()

predictions = []

for X in tqdm(test_iter):
    X = X.to(device)
    with torch.no_grad():
        logits = model(X)
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

preds = []
for i in predictions:
    preds.append(num_to_class[i])

test_data = pd.read_csv(test_path)
test_data['label'] = pd.Series(preds)
submission = pd.concat([test_data['image'], test_data['label']], axis=1)
submission.to_csv(saveFileName, index=False)
print("Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")

Finished reading the test set of Leaves Dataset (8800 samples found)


100%|██████████| 550/550 [04:34<00:00,  2.01it/s]

Done!!!!!!!!!!!!!!!!!!!!!!!!!!!
